Assignment 4:

In this module we’ll be looking at data from the New York City tree census:
https://data.cityofnewyork.us/Environment/2015-Street-Tree-Census-Tree-Data/uvpi-gqnh

This data is collected by volunteers across the city, and is meant to catalog information about every single tree in the city.

Build a dash app for a arborist studying the health of various tree species (as defined by the variable ‘spc_common’) across each borough (defined by the variable ‘borough’). This arborist would like to answer the following two questions for each species and in each borough:

What proportion of trees are in good, fair, or poor health according to the ‘health’ variable?

Are stewards (steward activity measured by the ‘steward’ variable) having an impact on the health of trees?

Please see the accompanying notebook for an introduction and some notes on the Socrata API.

Deployment: Dash deployment is more complicated than deploying shiny apps, so deployment in this case is optional (and will result in extra credit). You can read instructions on deploying a dash app to heroku here: https://dash.plot.ly/deployment

In [1]:
import pandas as pd
import numpy as np

In this module, we'll be looking at the New York City tree census. This data was provided by a volunteer driven census in 2015, and we'll be accessing it via the socrata API. The main site for the data is here, and on the upper right hand side you'll be able to see the link to the API.

The data is conveniently available in json format, so we should be able to just read it directly in to Pandas:

In [2]:
url = 'https://data.cityofnewyork.us/resource/nwxe-4ae8.json'
trees = pd.read_json(url)
trees.head(10)

,tree_id,block_id,created_at,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,spc_common,...,boro_ct,state,latitude,longitude,x_sp,y_sp,council_district,census_tract,bin,bbl
0,180683,348711,2015-08-27,3,0,OnCurb,Alive,Fair,Acer rubrum,red maple,...,4073900,New York,40.723092,-73.844215,1.027431e+06,202756.7687,29.0,739.0,4052307.0,4.022210e+09
1,200540,315986,2015-09-03,21,0,OnCurb,Alive,Fair,Quercus palustris,pin oak,...,4097300,New York,40.794111,-73.818679,1.034456e+06,228644.8374,19.0,973.0,4101931.0,4.044750e+09
2,204026,218365,2015-09-05,3,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,3044900,New York,40.717581,-73.936608,1.001823e+06,200716.8913,34.0,449.0,3338310.0,3.028870e+09
3,204337,217969,2015-09-05,10,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,3044900,New York,40.713537,-73.934456,1.002420e+06,199244.2531,34.0,449.0,3338342.0,3.029250e+09
4,189565,223043,2015-08-30,21,0,OnCurb,Alive,Good,Tilia americana,American linden,...,3016500,New York,40.666778,-73.975979,9.909138e+05,182202.4260,39.0,165.0,3025654.0,3.010850e+09
5,190422,106099,2015-08-30,11,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,1014500,New York,40.770046,-73.984950,9.884187e+05,219825.5227,3.0,145.0,1076229.0,1.011310e+09
6,190426,106099,2015-08-30,11,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,1014500,New York,40.770210,-73.985338,9.883112e+05,219885.2785,3.0,145.0,1076229.0,1.011310e+09
7,208649,103940,2015-09-07,9,0,OnCurb,Alive,Good,Tilia americana,American linden,...,1012700,New York,40.762724,-73.987297,9.877691e+05,217157.8561,3.0,133.0,1086093.0,1.010410e+09
8,209610,407443,2015-09-08,6,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,5006400,New York,40.596579,-74.076255,9.630732e+05,156635.5542,NaN,NaN,NaN,NaN
9,192755,207508,2015-08-31,21,0,OffsetFromCurb,Alive,Fair,Platanus x acerifolia,London planetree,...,3037402,New York,40.586357,-73.969744,9.926537e+05,152903.6306,47.0,37402.0,3320727.0,3.072350e+09


Looks good, but lets take a look at the shape of this data:

In [3]:
trees.shape

(1000, 45)

1,000 seems like too few trees for a city like New York, and a suspiciously round number. What's going on?

Socrata places a 1,000 row limit on their API. Raw data is meant to be "paged" through for applications, with the expectation that a UX wouldn't be able to handle a full dataset.

As a simple example, if we had a mobile app with limited space that only displayed trees 5 at a time, we could view the first 5 trees in the dataset with the url below:

In [4]:
firstfive_url = 'https://data.cityofnewyork.us/resource/nwxe-4ae8.json?$limit=5&$offset=0'
firstfive_trees = pd.read_json(firstfive_url)
firstfive_trees

,tree_id,block_id,created_at,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,spc_common,...,boro_ct,state,latitude,longitude,x_sp,y_sp,council_district,census_tract,bin,bbl
0,180683,348711,2015-08-27,3,0,OnCurb,Alive,Fair,Acer rubrum,red maple,...,4073900,New York,40.723092,-73.844215,1027431.148,202756.7687,29,739,4052307,4022210001
1,200540,315986,2015-09-03,21,0,OnCurb,Alive,Fair,Quercus palustris,pin oak,...,4097300,New York,40.794111,-73.818679,1034455.701,228644.8374,19,973,4101931,4044750045
2,204026,218365,2015-09-05,3,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,3044900,New York,40.717581,-73.936608,1001822.831,200716.8913,34,449,3338310,3028870001
3,204337,217969,2015-09-05,10,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,3044900,New York,40.713537,-73.934456,1002420.358,199244.2531,34,449,3338342,3029250001
4,189565,223043,2015-08-30,21,0,OnCurb,Alive,Good,Tilia americana,American linden,...,3016500,New York,40.666778,-73.975979,990913.775,182202.4260,39,165,3025654,3010850052


If we wanted the next 5, we would use this url:

In [5]:
nextfive_url = 'https://data.cityofnewyork.us/resource/nwxe-4ae8.json?$limit=5&$offset=5'
nextfive_trees = pd.read_json(nextfive_url)
nextfive_trees

,tree_id,block_id,created_at,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,spc_common,...,boro_ct,state,latitude,longitude,x_sp,y_sp,council_district,census_tract,bin,bbl
0,190422,106099,2015-08-30,11,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,1014500,New York,40.770046,-73.984950,988418.6997,219825.5227,3.0,145.0,1076229.0,1.011310e+09
1,190426,106099,2015-08-30,11,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,1014500,New York,40.770210,-73.985338,988311.1900,219885.2785,3.0,145.0,1076229.0,1.011310e+09
2,208649,103940,2015-09-07,9,0,OnCurb,Alive,Good,Tilia americana,American linden,...,1012700,New York,40.762724,-73.987297,987769.1163,217157.8561,3.0,133.0,1086093.0,1.010410e+09
3,209610,407443,2015-09-08,6,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,5006400,New York,40.596579,-74.076255,963073.1998,156635.5542,NaN,NaN,NaN,NaN
4,192755,207508,2015-08-31,21,0,OffsetFromCurb,Alive,Fair,Platanus x acerifolia,London planetree,...,3037402,New York,40.586357,-73.969744,992653.7253,152903.6306,47.0,37402.0,3320727.0,3.072350e+09


You can read more about paging using the Socrata API here.

In these docs, you'll also see more advanced functions (called SoQL) under the "filtering and query" section. These functions should be reminding you of SQL.

Think about the shape you want your data to be in before querying it. Using SoQL is a good way to avoid the limits of the API. For example, using the below query I can easily obtain the count of each species of tree in the Bronx:

In [6]:
boro = 'Bronx'
soql_url = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=spc_common,count(tree_id)' +\
        '&$where=boroname=\'Bronx\'' +\
        '&$group=spc_common').replace(' ', '%20')
soql_trees = pd.read_json(soql_url)

soql_trees

,count_tree_id,spc_common
0,4619,NaN
1,43,black walnut
2,9,spruce
3,62,tulip-poplar
4,7,trident maple
...,...,...
128,195,crimson king maple
129,774,purple-leaf plum
130,363,sawtooth oak
131,505,willow oak


This behavior is very common with web API's, and I think this is useful when thinking about building interactive data products. When in a Jupyter Notebook or RStudio, there's an expectation that (unless you're dealing with truly large datasets) the data you want can be brought into memory and manipulated.

Dash and Shiny abstract away the need to distinguish between the client side and server side to make web development more accessible to data scientists. This can lead to some unintentional design mistakes if you don't think about how costly your callback functions are (for example: nothing will stop you in dash from running a costly model triggered whenever a dropdown is called.)

The goal of using the Socrata is to force you to think about where your data operations are happening, and not resort to pulling in the data and performing all operations in local memory.

The code snippets provided above were extremely helpful, I used them to tackle the problems in assignment 4.

What proportion of trees are in good, fair, or poor health according to the ‘health’ variable? We need to calculate this for each species in each borough. I initially ran into errors and quickly realized that the variables in the select and group clause need to match otherwise you run into "HTTP Error 400: Bad Request" error.

In the below query, I first aggregate the count of trees by the "Health" variable which has three categories - "Fair", "Good" and "Poor". Health also happens to have a NaN category which I use to exclude rows in the where clause below. 

The other thing I figured out was how to assign an alias to the aggregated columns, this helped me calculate proportions in the final merged dataset. Otherwise, Python throws you an error when you're trying to calculate proportions using two columns that are named the same - 'count_tree_id'.

In [7]:
soql_url1 = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=boroname,spc_common,health,count(tree_id) as num_health' +\
        '&$where=health!=\'NaN\'' +\
        '&$group=boroname,spc_common,health').replace(' ', '%20')
soql_trees1 = pd.read_json(soql_url1)

soql_trees1

,boroname,spc_common,health,num_health
0,Bronx,American beech,Fair,7
1,Bronx,American beech,Good,21
2,Bronx,American beech,Poor,3
3,Bronx,American elm,Fair,248
4,Bronx,American elm,Good,1176
...,...,...,...,...
995,Manhattan,serviceberry,Good,30
996,Manhattan,serviceberry,Poor,5
997,Manhattan,shingle oak,Fair,34
998,Manhattan,shingle oak,Good,165


Next I need to aggregate the count of trees by species and by borough. Since I excluded rows with "NaN" above in the health column, I use the same filter in the where clause to ensure an apples-to-apples comparison.

For example, I see from below query that there are 31 American Beech trees in Bronx - 7 in "Fair" condition, 21 in "Good" condition and the remaining 3 in "Poor" condition (from query 1 above). I can now calculate proportions by merging the datasets.

In [8]:
soql_url2 = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=boroname,spc_common, count(tree_id) as num_total' +\
        '&$where=health!=\'NaN\'' +\
        '&$group=boroname,spc_common').replace(' ', '%20')
soql_trees2 = pd.read_json(soql_url2)

soql_trees2

,boroname,spc_common,num_total
0,Bronx,American beech,31
1,Bronx,American elm,1471
2,Bronx,American hophornbeam,185
3,Bronx,American hornbeam,295
4,Bronx,American larch,12
...,...,...,...
652,Staten Island,weeping willow,55
653,Staten Island,white ash,155
654,Staten Island,white oak,232
655,Staten Island,white pine,47


The merged dataset below now shows the total count of trees grouped by the "Health" variable and the total number of trees by species within each borough. So, we can easily calculate the proportion of trees by "Health" category for each species within each borough as requested in question 1.

In [9]:
soql_merged1 = pd.merge(soql_trees1, soql_trees2, on=['boroname', 'spc_common']) 

soql_merged1.columns = ['boroname', 'spc_common', 'health', 'num_health', 'num_total']

soql_merged1.head(1000)

,boroname,spc_common,health,num_health,num_total
0,Bronx,American beech,Fair,7,31
1,Bronx,American beech,Good,21,31
2,Bronx,American beech,Poor,3,31
3,Bronx,American elm,Fair,248,1471
4,Bronx,American elm,Good,1176,1471
...,...,...,...,...,...
995,Manhattan,serviceberry,Good,30,38
996,Manhattan,serviceberry,Poor,5,38
997,Manhattan,shingle oak,Fair,34,205
998,Manhattan,shingle oak,Good,165,205


In [10]:
soql_merged1['prop_health'] = soql_merged1['num_health'] / soql_merged1['num_total']

soql_merged1.sort_values(by=['boroname','spc_common','health'])

q1 = soql_merged1[['boroname','spc_common','health','prop_health']]

q1.head(10)

,boroname,spc_common,health,prop_health
0,Bronx,American beech,Fair,0.225806
1,Bronx,American beech,Good,0.677419
2,Bronx,American beech,Poor,0.096774
3,Bronx,American elm,Fair,0.168593
4,Bronx,American elm,Good,0.799456
5,Bronx,American elm,Poor,0.031951
6,Bronx,American hophornbeam,Fair,0.151351
7,Bronx,American hophornbeam,Good,0.800000
8,Bronx,American hophornbeam,Poor,0.048649
9,Bronx,American hornbeam,Fair,0.206780


As we can see from the sample above, 67.7% of American Beeches in Bronx are in "Good" condition while only 9.7% are in "Poor" condition etc. 

The second question we are trying to answer is "Are stewards (steward activity measured by the ‘status’ variable) having an impact on the health of trees?"

From the data dictionary, one would expect acts of stewardship such as adding tree guards, mulch, seating etc. to improve the health of a tree and vice versa. Quantitatively, one would expect no acts of stewardship or value of "None" for the "Steward" column to correspond either with "Poor" health of trees or dead stumps and vice versa a value of "1or2" or "3or4" more acts of stewardship to correspond with generally improved health represented either by "Fair" or "Good" health of trees.

We can calculate these proportions just as we did for question 1 to confirm this hypothesis.

In [11]:
#This gives the total count by steward category for each species in each borough

soql_url3 = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=boroname,spc_common,health,steward,count(tree_id) as steward_total' +\
        '&$where=health!=\'NaN, steward!=None\'' +\
        '&$group=boroname,spc_common,health,steward').replace(' ', '%20')

soql_trees3 = pd.read_json(soql_url3)

soql_trees3

,boroname,spc_common,health,steward,steward_total
0,Bronx,American beech,Fair,1or2,3
1,Bronx,American beech,Fair,3or4,1
2,Bronx,American beech,Fair,None,3
3,Bronx,American beech,Good,1or2,2
4,Bronx,American beech,Good,3or4,1
...,...,...,...,...,...
995,Brooklyn,blue spruce,Good,None,5
996,Brooklyn,blue spruce,Poor,None,1
997,Brooklyn,boxelder,Fair,1or2,1
998,Brooklyn,boxelder,Fair,None,1


In [12]:
#Here we extract count of trees by stewardship category by species name and by borough where stewardship is "None"

soql_url4 = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=boroname,spc_common,health,steward, count(tree_id)' +\
        '&$where=steward=\'None\'' +\
        '&$group=boroname,spc_common,health, steward').replace(' ', '%20')

soql_trees4 = pd.read_json(soql_url4)

soql_trees4

,boroname,spc_common,health,steward,count_tree_id
0,Bronx,American beech,Fair,None,3
1,Bronx,American beech,Good,None,18
2,Bronx,American beech,Poor,None,2
3,Bronx,American elm,Fair,None,181
4,Bronx,American elm,Good,None,927
...,...,...,...,...,...
995,Queens,American beech,Fair,None,5
996,Queens,American beech,Good,None,46
997,Queens,American beech,Poor,None,6
998,Queens,American elm,Fair,None,170


In [13]:
#We merge the dataset where stewardship is 'None' with the dataset that contains the total count of trees by "Health" 
#category so we can calculate proportions.

soql_merged2 = pd.merge(soql_merged1, soql_trees4, on=['boroname', 'spc_common','health'])

soql_merged2.columns = ['boroname', 'spc_common','health','num_health', 'num_total','prop_health','steward','count_tree_id']

soql_merged2.head(1000)

,boroname,spc_common,health,num_health,num_total,prop_health,steward,count_tree_id
0,Bronx,American beech,Fair,7,31,0.225806,None,3
1,Bronx,American beech,Good,21,31,0.677419,None,18
2,Bronx,American beech,Poor,3,31,0.096774,None,2
3,Bronx,American elm,Fair,248,1471,0.168593,None,181
4,Bronx,American elm,Good,1176,1471,0.799456,None,927
...,...,...,...,...,...,...,...,...
934,Manhattan,serviceberry,Good,30,38,0.789474,None,11
935,Manhattan,serviceberry,Poor,5,38,0.131579,None,2
936,Manhattan,shingle oak,Fair,34,205,0.165854,None,16
937,Manhattan,shingle oak,Good,165,205,0.804878,None,106


In [14]:
#Calculate proportions where stewardship is "None" across health categories.

soql_merged2['prop_steward'] = soql_merged2['count_tree_id'] / soql_merged2['num_health']

soql_merged2.sort_values(by=['boroname','spc_common','health'])

q2 = soql_merged2[['boroname','spc_common','health','steward','prop_steward']]

q2

,boroname,spc_common,health,steward,prop_steward
0,Bronx,American beech,Fair,None,0.428571
1,Bronx,American beech,Good,None,0.857143
2,Bronx,American beech,Poor,None,0.666667
3,Bronx,American elm,Fair,None,0.729839
4,Bronx,American elm,Good,None,0.788265
...,...,...,...,...,...
934,Manhattan,serviceberry,Good,None,0.366667
935,Manhattan,serviceberry,Poor,None,0.400000
936,Manhattan,shingle oak,Fair,None,0.470588
937,Manhattan,shingle oak,Good,None,0.642424


We can see from above that even though there was no stewardship for 18 of the 21 (85.7%) American Beeches in Bronx they were classified to be in "Good" condition. Similarly, there was no stewardship for 3 of the 7 (42.8%) American Beeches in Bronx that were classified to be in "Fair" condition. On the other hand, trees across boroughs that were in "Poor" condition consistently demonstrated no stewardship - 66.6% fo American Beeches in Bronx, 40% of serviceberry in Manhattan, 50% of shingle oak in Manhattan were all classified to be in "Poor" health and coincidentally demonstrated no stewardship.

So, there seems to be insufficient evidence that points to the lack of stewardship resulting in trees being in "Poor" health, other factors may be at play.


In [15]:
#We replicate the analysis above for some stewardship

soql_url5 = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=boroname,spc_common,health,steward, count(tree_id)' +\
        '&$where=steward!=\'None\'' +\
        '&$group=boroname,spc_common,health, steward').replace(' ', '%20')

soql_trees5 = pd.read_json(soql_url5)

soql_trees5

,boroname,spc_common,health,steward,count_tree_id
0,Bronx,American beech,Fair,1or2,3
1,Bronx,American beech,Fair,3or4,1
2,Bronx,American beech,Good,1or2,2
3,Bronx,American beech,Good,3or4,1
4,Bronx,American beech,Poor,1or2,1
...,...,...,...,...,...
995,Brooklyn,sassafras,Good,3or4,2
996,Brooklyn,sassafras,Poor,1or2,1
997,Brooklyn,sawtooth oak,Fair,1or2,24
998,Brooklyn,sawtooth oak,Fair,3or4,6


In [16]:
#We merge the dataset where stewardship is not 'None' with the dataset that contains the total count of trees by "Health" 
#category so that we can calculate proportions.

soql_merged3 = pd.merge(soql_merged1, soql_trees5, on=['boroname', 'spc_common','health'])

soql_merged3.columns = ['boroname', 'spc_common','health','num_health', 'num_total','prop_health','steward','count_tree_id']

soql_merged3.head(1000)

,boroname,spc_common,health,num_health,num_total,prop_health,steward,count_tree_id
0,Bronx,American beech,Fair,7,31,0.225806,1or2,3
1,Bronx,American beech,Fair,7,31,0.225806,3or4,1
2,Bronx,American beech,Good,21,31,0.677419,1or2,2
3,Bronx,American beech,Good,21,31,0.677419,3or4,1
4,Bronx,American beech,Poor,3,31,0.096774,1or2,1
...,...,...,...,...,...,...,...,...
995,Brooklyn,sassafras,Good,35,41,0.853659,3or4,2
996,Brooklyn,sassafras,Poor,2,41,0.048780,1or2,1
997,Brooklyn,sawtooth oak,Fair,104,785,0.132484,1or2,24
998,Brooklyn,sawtooth oak,Fair,104,785,0.132484,3or4,6


In [17]:
#Calculate proportions where stewardship is not "None" across health categories.

soql_merged3['prop_steward'] = soql_merged3['count_tree_id'] / soql_merged3['num_health']

soql_merged3.sort_values(by=['boroname','spc_common','health'])

q3 = soql_merged3[['boroname','spc_common','health','steward','prop_steward']]

q3

,boroname,spc_common,health,steward,prop_steward
0,Bronx,American beech,Fair,1or2,0.428571
1,Bronx,American beech,Fair,3or4,0.142857
2,Bronx,American beech,Good,1or2,0.095238
3,Bronx,American beech,Good,3or4,0.047619
4,Bronx,American beech,Poor,1or2,0.333333
...,...,...,...,...,...
995,Brooklyn,sassafras,Good,3or4,0.057143
996,Brooklyn,sassafras,Poor,1or2,0.500000
997,Brooklyn,sawtooth oak,Fair,1or2,0.230769
998,Brooklyn,sawtooth oak,Fair,3or4,0.057692


Build app using dash:

In [18]:
#!pip install --upgrade plotly
#!pip install jupyter-dash

In [ ]:
#Display results of the proportion of trees categorized by health by species and by borough

import dash
import dash_core_components as dcc
import dash_html_components as html

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

df = q1

available_indicators = df['spc_common'].unique()

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)



app.layout = html.Div([
    html.H1('The proportion of trees categorized by health by species and by borough'),
    html.Div('''
        spc_common
    '''),
    dcc.Dropdown(
        id='my-dropdown',
        options=[{'label': i, 'value': i} for i in available_indicators],
        value='Atlas cedar'
    ),
    dcc.Graph(
        id='example-graph'    
    )
    
])

@app.callback(
    dash.dependencies.Output('example-graph', 'figure'),
    [dash.dependencies.Input('my-dropdown', 'value')])

def update_output(selected_dropdown_value):
    dff = df[df['spc_common'] == selected_dropdown_value]
    figure = {
            'data': [
                {'x': dff.boroname[dff['health'] == 'Good'], 'y': dff.prop_health[dff['health'] == 'Good'], 'type': 'bar', 'name': 'Good'},
                {'x': dff.boroname[dff['health'] == 'Fair'], 'y': dff.prop_health[dff['health'] == 'Fair'], 'type': 'bar', 'name': 'Fair'},
                {'x': dff.boroname[dff['health'] == 'Poor'], 'y': dff.prop_health[dff['health'] == 'Poor'], 'type': 'bar', 'name': 'Poor'}
            ],
            'layout': {
                'title': 'Prop. of health by species by borough name'
            }
        }
    return figure 


if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


<ipython-input-19-2737a05ee23c>:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
<ipython-input-19-2737a05ee23c>:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Sep/2022 19:24:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2022 19:24:36] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2022 19:24:36] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2022 19:24:36] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2022 19:24:36] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2022 19:24:36] "POST /_dash-update-component HTTP/1.1" 200 -
1

In [ ]:
#Display results of the proportion of health for "No Stewardship" by species by borough

import dash
import dash_core_components as dcc
import dash_html_components as html

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

df = q2

available_indicators = df['spc_common'].unique()

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)



app.layout = html.Div([
    html.H1('The proportion of health for "No Stewardship" by species by borough'),
    html.Div('''
        spc_common
    '''),
    dcc.Dropdown(
        id='my-dropdown',
        options=[{'label': i, 'value': i} for i in available_indicators],
        value='Atlas cedar'
    ),
    dcc.Graph(
        id='example-graph'    
    )
    
])

@app.callback(
    dash.dependencies.Output('example-graph', 'figure'),
    [dash.dependencies.Input('my-dropdown', 'value')])

def update_output(selected_dropdown_value):
    dff = df[df['spc_common'] == selected_dropdown_value]
    figure = {
            'data': [
                {'x': dff.boroname[dff['health'] == 'Good'], 'y': dff.prop_steward[dff['health'] == 'Good'], 'type': 'bar', 'name': 'Good'},
                {'x': dff.boroname[dff['health'] == 'Fair'], 'y': dff.prop_steward[dff['health'] == 'Fair'], 'type': 'bar', 'name': 'Fair'},
                {'x': dff.boroname[dff['health'] == 'Poor'], 'y': dff.prop_steward[dff['health'] == 'Poor'], 'type': 'bar', 'name': 'Poor'}
            ],
            'layout': {
                'title': 'The proportion of health for "No Stewardship"'
            }
        }
    return figure 


if __name__ == '__main__':
    app.run_server()

In [ ]:
#Display results of the proportion of health when some stewardship is provided by species by borough

import dash
import dash_core_components as dcc
import dash_html_components as html

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

df = q3

available_indicators = df['spc_common'].unique()

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)



app.layout = html.Div([
    html.H1('The proportion of health when some stewardship is provided by species by borough'),
    html.Div('''
        spc_common
    '''),
    dcc.Dropdown(
        id='my-dropdown',
        options=[{'label': i, 'value': i} for i in available_indicators],
        value='Atlas cedar'
    ),
    dcc.Graph(
        id='example-graph'    
    )
    
])

@app.callback(
    dash.dependencies.Output('example-graph', 'figure'),
    [dash.dependencies.Input('my-dropdown', 'value')])

def update_output(selected_dropdown_value):
    dff = df[df['spc_common'] == selected_dropdown_value]
    figure = {
            'data': [
                {'x': dff.health[dff['steward'] == '1or2'], 'y': dff.prop_steward[dff['steward'] == '1or2'], 'type': 'bar', 'name': '1or2'},
                {'x': dff.health[dff['steward'] == '3or4'], 'y': dff.prop_steward[dff['steward'] == '3or4'], 'type': 'bar', 'name': '3or4'},
                {'x': dff.health[dff['steward'] == '4orMore'], 'y': dff.prop_steward[dff['steward'] == '4orMore'], 'type': 'bar', 'name': '4orMore'}
            ],
            'layout': {
                'title': 'Prop. of Steward by health'
            }
        }
    return figure 

if __name__ == '__main__':
    app.run_server()

In [ ]:
#Dash app to display the impact of some stewardship on health by borough

import dash
import dash_core_components as dcc
import dash_html_components as html

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

df = q3

available_indicators = df['boroname'].unique()

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)



app.layout = html.Div([
    html.H1('The proportion of health when some stewardship is provided by borough'),
    html.Div('''
        boroname
    '''),
    dcc.Dropdown(
        id='my-dropdown',
        options=[{'label': i, 'value': i} for i in available_indicators],
        value='Queens'
    ),
    dcc.Graph(
        id='example-graph'    
    )
    
])

@app.callback(
    dash.dependencies.Output('example-graph', 'figure'),
    [dash.dependencies.Input('my-dropdown', 'value')])

def update_output(selected_dropdown_value):
    dff = df[df['boroname'] == selected_dropdown_value]
    figure = {
            'data': [
                {'x': dff.health[dff['steward'] == '1or2'], 'y': dff.prop_steward[dff['steward'] == '1or2'], 'type': 'bar', 'name': '1or2'},
                {'x': dff.health[dff['steward'] == '3or4'], 'y': dff.prop_steward[dff['steward'] == '3or4'], 'type': 'bar', 'name': '3or4'},
                {'x': dff.health[dff['steward'] == '4orMore'], 'y': dff.prop_steward[dff['steward'] == '4orMore'], 'type': 'bar', 'name': '4orMore'}
            ],
            'layout': {
                'title': 'Prop. of Steward by health'
            }
        }
    return figure 


if __name__ == '__main__':
    app.run_server()